# Anonmymization

In [14]:
import hashlib as hl

string_to_hash = "supersäkert_lösen0rd".encode()
string_to_hash2 = "Supersäkert_lösen0rd".encode()
string_to_hash3 = "Supersäkert_lösen0rd".encode()
string_to_hash4 = "Supersäkert_lösen0rd".encode()

hl.sha256(string_to_hash).hexdigest(), hl.sha256(string_to_hash2).hexdigest()

('919508f6494495c4bbebc2c87f3dee7b9245bebcd462ec1591fedc5e7fca0386',
 '98b8b43ac8e49f673fd150ebf3d8c39fc4f5a76edfee2b4150a17757d2f58743')

In [13]:
hl.sha224(string_to_hash3).hexdigest()

'5b5439db6af04fd3097a8c3c8557fd366adde92ac94952b83ede3ac5'

In [15]:
hl.md5(string_to_hash4).hexdigest()

'c1b867aa071e0c43714f70c246855987'

# Example with Pandas

In [16]:
import pandas as pd

df = pd.read_csv("../Data/BankChurners.csv", usecols = ["CLIENTNUM", "Attrition_Flag", "Customer_Age", "Gender", "Total_Trans_Amt"])

df.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Total_Trans_Amt
0,768805383,Existing Customer,45,M,1144
1,818770008,Existing Customer,49,F,1291
2,713982108,Existing Customer,51,M,1887
3,769911858,Existing Customer,40,F,1171
4,709106358,Existing Customer,40,M,816


In [17]:
df['CLIENTNUM'] = df['CLIENTNUM'].astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CLIENTNUM        10127 non-null  object
 1   Attrition_Flag   10127 non-null  object
 2   Customer_Age     10127 non-null  int64 
 3   Gender           10127 non-null  object
 4   Total_Trans_Amt  10127 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 395.7+ KB


In [21]:
hashes_clientum = df['CLIENTNUM'].apply(lambda x: hl.sha256(x.encode()).hexdigest())

- Lägger til en ny kolumn som har hashen

In [22]:
df.insert(1, "Hash_clientum", hashes_clientum)
df.head()

,CLIENTNUM,Hash_clientum,Attrition_Flag,Customer_Age,Gender,Total_Trans_Amt
0,768805383,c9bbef56f9d8292cb3cfa8ae91f9b9167390e6e4b514d5...,Existing Customer,45,M,1144
1,818770008,7996e2340d70489252370a5df035ec99381c8344cc3511...,Existing Customer,49,F,1291
2,713982108,6fb53dbc743724e086243b5bc288df62b4a6dc1b8bde92...,Existing Customer,51,M,1887
3,769911858,f86b86a1047317685f29c399059b199858685faf5ec6a8...,Existing Customer,40,F,1171
4,709106358,0d239470b0cb57e110cf60bc3865344ee2cdced6e3acdc...,Existing Customer,40,M,816


# High performance Pandas
´´´py
mask = (x > .5) & (y < .5)
df[df[mask]]
```

under the hood

```py
tmp1 = (x > .5)
tmp2 = (y < .5)
mask = tmp1 & tmp2
```

use pd.eval("") -> does elementwise operation with numexpr

In [24]:
import numpy as np
import pandas as pd

nrows, ncols = 1_000_000, 100

df1, df2, df3, df4 = [pd.DataFrame(np.random.randn(nrows, ncols)) for _ in range(4)]

In [30]:
%timeit df1 + df2 + df3 + df4
%timeit pd.eval("df1+df2+df3+df4")

1.07 s ± 33.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
985 ms ± 18.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
standard = df1+df2+df3+df4
sum_eval = pd.eval("df1+df2+df3+df4")

sum_eval.equals(standard)

True

In [36]:
rolls = pd.DataFrame(np.random.randint(1,6, (6,3)), columns = ["Die1", "Die2", "Die3"])
rolls

,Die1,Die2,Die3
0,1,3,4
1,4,3,2
2,1,5,4
3,2,2,1
4,1,3,1
5,5,1,4


In [40]:
rolls.eval("Sum = Die1 + Die2 + Die3", inplace=True)

In [49]:
high = 9
rolls.eval("Winner = Sum > @high")

,Die1,Die2,Die3,Sum,Winner
0,1,3,4,8,False
1,4,3,2,9,False
2,1,5,4,10,True
3,2,2,1,5,False
4,1,3,1,5,False
5,5,1,4,10,True


In [59]:
rolls[rolls["Sum"] > high] # Det tradionella sättet att filtrera på

,Die1,Die2,Die3,Sum
2,1,5,4,10
5,5,1,4,10


# Query 

- filter using query

In [65]:
rolls.query("Sum > @high")
# Man tjänar tid att använda query och eval ifall du har stora dataframes
rolls.query("Sum > @high & Die1 == 1")

,Die1,Die2,Die3,Sum
2,1,5,4,10
